In [ ]:
from src.data import RawDataset, Dataset
from src import workflow

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import logging
from src.logging import logger
logger.setLevel(logging.DEBUG)

# Raw Data is Read Only: Sing it with me!

How do you asseble raw data into a usable dataset? How do you make sure your raw datasets are reproducible?
We are going to work through the process of collecting raw data files (using an object called a `RawDataset`), and converting then into something useful for our reproducible data science workflow (the `Dataset` object)

## Bjørn's Problem: Supervised Learning

Bjørn employs a large number of Finnish line cooks. He can’t understand a word they say.

Bjørn needs a trained model to do real-time translation from Finnish to Swedish.

Bjørn has decided to start with the Finnish phoneme dataset shipped with a project called lvq-pak. His objective is to train three different models, and choose the one with the best overall accuracy score.


### LVQ-Pak: A Finnish Phonetic dataset

The Learning Vector Quantization (lvq-pak) project includes a simple Finnish phonetic dataset
consisting 20-dimensional Mel Frequency Cepstrum Coefficients (MFCCs) labelled with target phoneme information. Our goal is to explore this dataset, process it into a useful form, and make it useful for reproducible data science

In [ ]:
dataset_name='lvq-pak'  # Naming things: the hardest problem in computer science.

lvq-pak is shipped as a source code tarball. The data files are included as textfiles within that download. Our first goal is to retrieve these source files, and record some metadata about them.

###  Grab the source code package
We can add a file to the RawDataset using its `add_url()` method. If we know the hash of this file, we should include it here, and it will be checked on fownload. If not, one will be computed from this download and used for comparison on subsequent downloads.


In [ ]:
lvq_pak = RawDataset(dataset_name)
lvq_pak.add_url(url="http://www.cis.hut.fi/research/lvq_pak/lvq_pak-3.1.tar")
print(lvq_pak)

There are some additional options to `fetch_url()` we may wish to use

In [ ]:
help(RawDataset.add_url)

`hash_value` and `hash_type` should be self-evident.

`file_name` is optional. If skipped, the name will be guessed from the URL. It's often nice to specify, however, Use this attribute to override this guess.

The `name` field can either be used to specify a text label to describe the dataset. It can also be used to tag 2 special pieces of metadata: `DESCR` and `LICENSE`, the downloaded (text) file will be used as the dataset description and license text, respectively.

We will use this feature to add a README to our dataset:

In [ ]:
lvq_pak.add_url(url='http://www.cis.hut.fi/research/lvq_pak/README',
                file_name=f'{dataset_name}.readme',
                name='DESCR')

Datasets should *always* have an explicit license. Reading the project documentation, we see a license in one of the textfiles. We can extract and use it via the `add_metadata()` method

In [ ]:
license_txt = '''
************************************************************************
*                                                                      *
*                              LVQ_PAK                                 *
*                                                                      *
*                                The                                   *
*                                                                      *
*                   Learning  Vector  Quantization                     *
*                                                                      *
*                          Program  Package                            *
*                                                                      *
*                   Version 3.1 (April 7, 1995)                        *
*                                                                      *
*                          Prepared by the                             *
*                    LVQ Programming Team of the                       *
*                 Helsinki University of Technology                    *
*           Laboratory of Computer and Information Science             *
*                Rakentajanaukio 2 C, SF-02150 Espoo                   *
*                              FINLAND                                 *
*                                                                      *
*                      Copyright (c) 1991-1995                         *
*                                                                      *
************************************************************************
*                                                                      *
*  NOTE: This program package is copyrighted in the sense that it      *
*  may be used for scientific purposes. The package as a whole, or     *
*  parts thereof, cannot be included or used in any commercial         *
*  application without written permission granted by its producents.   *
*  No programs contained in this package may be copied for commercial  *
*  distribution.                                                       *
*                                                                      *
*  All comments concerning this program package may be sent to the     *
*  e-mail address 'lvq@nucleus.hut.fi'.                                *
*                                                                      *
************************************************************************
'''
lvq_pak.add_metadata(contents=license_txt, kind='LICENSE')

### Fetching the Data

In [ ]:
lvq_pak.fetch()

In [ ]:
from src.utils import list_dir
from src.paths import raw_data_path
list_dir(raw_data_path)

## Processing the data
The next step is to write the importer that actually processes the data we will be using for this dataset.

The important things to generate are `data` and `target` entries. A `metadata` is optional, but recommended if you want to save additional information about the dataset.

Usually, this functionality gets bundled up into a function and added to `datasets.py`


In [ ]:
# Unpack the file
untar_dir = lvq_pak.unpack()
list_dir(untar_dir)

In [ ]:
unpack_dir = untar_dir / 'lvq_pak-3.1'
list_dir(unpack_dir)

In this dataset, the training and test datsets are stored in files named `ex1.dat` and `ex2.dat` respectively

In [ ]:

datafile_train = unpack_dir / 'ex1.dat'
datafile_test = unpack_dir / 'ex2.dat'

datafile_train.exists() and datafile_test.exists()

According to the documentation, the data format is space-delimited, with the class label included as the last column. Let's have a look

In [ ]:
print(head_file(datafile_train))

Indeed, the datafile consists of 1 line containing the dimension of the data, a comment, and then 21 space-delimited columns, the final column being the target class label. 

**Note:** We have to be a little careful importing the data, because '#' is used both as the comment delimiter, and as a class label.

Fortunately, we have a helper function for this. We will get a little cheeky and skip the first 2 lines (hoping there are no other comments). The documentation also says ther are 1962 entries in each of the training and test datasets.

In [ ]:
def read_space_delimited(filename, skiprows=None, class_labels=True):
    """Read an space-delimited file

    skiprows: list of rows to skip when reading the file.

    Note: we can't use automatic comment detection, as
    `#` characters are also used as data labels.
    class_labels: boolean
        if true, the last column is treated as the class label
    """
    with open(filename, 'r') as fd:
        df = pd.read_table(fd, skiprows=skiprows, skip_blank_lines=True, comment=None, header=None, sep=' ', dtype=str)
        # targets are last column. Data is everything else
        if class_labels is True:
            target = df.loc[:, df.columns[-1]].values
            data = df.loc[:, df.columns[:-1]].values
        else:
            data = df.values
            target = np.zeros(data.shape[0])
        return data, target


In [ ]:
data, target = read_space_delimited(datafile_train, skiprows=[0,1])
data2, target2 = read_space_delimited(datafile_test, skiprows=[0])

data.shape, target.shape, data2.shape, target2.shape

In [ ]:
target

This seems to work, so let's wrap this functionality up into a processing function.
By convention, the function takes a `dataset_name`, and any other options that may be useful for reading the data, and returns a dictionary that matches the `Dataset` constructor signature.

We will place this function in `localdata.py`, (and add it to `__all__`) to make it visible to our dataset code.

In [ ]:
#%%file ../src/data/localdata.py
"""
Custom dataset processing/generation functions should be added to this file
"""

from src.data.utils import read_space_delimited, normalize_labels
from src.paths import interim_data_path
import numpy as np

__all__ = ['process_lvq_pak']

def process_lvq_pak(dataset_name='lvq-pak', kind='all', numeric_labels=True, metadata=None):
    """
    kind: {'test', 'train', 'all'}, default 'all'
    numeric_labels: boolean (default: True)
        if set, target is a vector of integers, and label_map is created in the metadata
        to reflect the mapping to the string targets
    """
    
    untar_dir = interim_data_path / dataset_name
    unpack_dir = untar_dir / 'lvq_pak-3.1'

    if kind == 'train':
        data, target = read_space_delimited(unpack_dir / 'ex1.dat', skiprows=[0,1])
    elif kind == 'test':
        data, target = read_space_delimited(unpack_dir / 'ex2.dat', skiprows=[0])
    elif kind == 'all':
        data1, target1 = read_space_delimited(unpack_dir / 'ex1.dat', skiprows=[0,1])
        data2, target2 = read_space_delimited(unpack_dir / 'ex2.dat', skiprows=[0])
        data = np.vstack((data1, data2))
        target = np.append(target1, target2)
    else:
        raise Exception(f'Unknown kind: {kind}')

    if numeric_labels:
        if metadata is None:
            metadata = {}
        mapped_target, label_map = normalize_labels(target)
        metadata['label_map'] = label_map
        target = mapped_target

    dset_opts = {
        'dataset_name': dataset_name,
        'data': data,
        'target': target,
        'metadata': metadata
    }
    return dset_opts


Let's make sure this works as expected

In [ ]:
from src.data.localdata import process_lvq_pak

for kind in ['train', 'test', 'all']:
    dset_opts = process_lvq_pak(kind=kind)
    dset = Dataset(**dset_opts)
    print(f'{kind}: data={dset.data.shape} target={dset.target.shape}')

This all looks good. Add this function to the `RawDataset`, and add the `RawDataset` to the global dataset list


In [ ]:
lvq_pak.load_function = process_lvq_pak

In [ ]:
workflow.add_raw_dataset(lvq_pak)

Finally, re-load the dataset and save a copy of it

In [ ]:
lvq = Dataset.from_raw(dataset_name, force=True)
print(str(lvq))

Alternately, we can create an empty transformer to convert this RawDataset into a Dataset and save it

In [ ]:
workflow.add_transformer(from_raw=dataset_name)

In [ ]:
workflow.get_transformer_list()

The complete dataset cam be written to `processed_data_path`. A copy of just the metadata is also stored, so that it may be quickly checked without loading the entire dataset:

In [ ]:
logger.setLevel(logging.INFO)
workflow.apply_transforms()

In [ ]:
lvq_meta = Dataset.load(dataset_name, metadata_only=True)

In [ ]:
type(lvq_meta)

In [ ]:
print(lvq.DATA_HASH)

## Train/Test Split
It's fairly straightforward to create and split a dataset into training and test sets:

In [ ]:
workflow.available_datasets()

In [ ]:
workflow.available_transformers()

In [ ]:
workflow.get_transformer_list()

In [ ]:
transform_pipeline = [("train_test_split", {'random_state':6502, 'test_size':0.2})]
workflow.add_transformer(from_raw='lvq-pak', suppress_output=True, transformations=transform_pipeline)

In [ ]:
logger.setLevel(logging.INFO) # This step is a bit noisy, otherwise.
workflow.apply_transforms()
logger.setLevel(logging.DEBUG)

Notice there are now 2 new datasets: `lvq-pak_test` and `lvq-pak_train`

In [ ]:
workflow.available_datasets()

We can now load the split data. The metadata now reflects the options that were used to generate this split.

In [ ]:
ds = Dataset.load('lvq-pak_train')
ds.SPLIT_OPTS